# **Procura de Problemas Éticos em Issues**

## **Imports necessários para o funcionamento:**

In [ ]:
%pip install pandas
%pip install nltk
%pip install ipywidgets
%pip install seaborn
%pip install matplotlib
%pip install networkx
%pip install plotly
%pip install wordcloud


import pandas as pd
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import ipywidgets as widgets
from IPython.display import display, HTML
from collections import Counter
import networkx as nx
import numpy as np

## **Fazendo a leitura dos dados:**

In [ ]:
issues = pd.read_csv("out/llama-issues-3.csv")
comentarios = pd.read_csv("out/llama-comentarios-3.csv")

# Concatenando o Titulo e a Descrição das issues
issues['Info'] = issues['TituloIssue'] + issues['DescricaoIssue']

# Convertendo todos os tipos para Strings minusculas
issues['Info'] = issues['Info'].astype(str).str.lower()
comentarios['Comentario'] = comentarios['Comentario'].astype(str).str.lower()

## **Tokenizando os Comentarios, Titulo e Descrição das Issues**

In [ ]:
# Definindo as StopWords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

# Função que remove as stopwords
def removerStopWords(palavras):
    return [palavra for palavra in palavras if palavra not in stopWords and palavra.isalpha() or palavra in expressoes]

# Criando um tokenizer que considera expressões
expressoes = {'data governance', 'data protection', 'informed consent', 'lack of data', 'user data collection', 'conflict of interest', 'human agency', 'intellectual property',
              'regulatory approaches', 'professional ethics', 'work ethics', 'common goods', 'individual differences', 'prevention of harm', 'quality of life',
              'respect for human autonomy', 'retention and addiction', 'social justice', 'speech issues', 'technical robustness', 'computer abuse', 'malicious use', 'context of use',
              'self respect', 'harmful content', 'interaction risks', 'labor displacement', 'economic impact', 'unethical actions', 'lack of empaty', 'renewable energy', 'moral views'}

expressaoRegular = r'\b(?:' + '|'.join(map(re.escape,expressoes)) + r')\b|\w+'
tokenizer = RegexpTokenizer(expressaoRegular)



# Tokenize dos comentarios das issues
comentariosToK = []
for i in range (comentarios['Comentario'].size):
    comentariosToK.append(removerStopWords(tokenizer.tokenize(comentarios.loc[i]['Comentario'])))

# Tokenize do campo Info
tituloDescricaoToK = []
for k in range(issues['Info'].size):
    tituloDescricaoToK.append(removerStopWords(tokenizer.tokenize(issues.loc[k]['Info'])))

## **Preparando o Dataframe com os termos relacionados à ética:**

In [ ]:
#Definindo os EthicalIssues

def toLower(palavras):
    return [palavra.lower() for palavra in palavras]


TaxonomiaOtavio = ['Bias', 'Data Governance', 'Data Protection', 'Encryption', 'Informed Consent', 'Lack of Data', 'Monetization', 'Openness', 'Privacy', 'User Data Collection'
                   'Authorship', 'Autonomy', 'Beneficence', 'Business', 'Ethics', 'Commerce', 'Compliance', 'Confidentiality', 'Conflict of Interest', 'Context', 'Dependability'
                   'Fairness', 'Human Agency', 'Intellectual Property', 'Oversight', 'Regulatory Approaches', 'Responsibility', 'Trust', 'Trustworthiness', 'Axiology', 'Freedom'
                   'Self-Conception', 'Solidarity', 'Utility', 'Care', 'Competence', 'Professional Ethics', 'Work Ethics', 'Access', 'Accessibility', 'Common Goods', 'Dignity',
                   'Diversity', 'Equality', 'Equity', 'Humanity', 'Inclusiveness', 'Individual Differences', 'Inequality', 'Justice', 'Non-Discrimination', 'Non-Maleficence',
                   'Participation', 'Plurality', 'Prevention of Harm', 'Quality of Life', 'Respect for Human Autonomy', 'Retention and Addiction', 'Social Justice', 'Sustainability',
                   'Unemployment', 'Welfare', 'Accountability', 'Accuracy', 'Anonymity', 'Comprehensibility', 'Consistency', 'Contestability', 'Explainability', 'Explicability',
                   'Integrity', 'Interpretability', 'Liability', 'Reliability', 'Safety', 'Security', 'Speech Issues', 'Technical Robustness', 'Traceability', 'Transparency',
                   'Usability', 'Computer Abuse', 'Malicious Use']
TaxonomiaOtavio = toLower(TaxonomiaOtavio)

OtavioThesaurus = ['preconception', 'prejudice', 'encoding', 'receptiveness', 'receptivity', 'privateness', 'secrecy', 'self-sufficiency', 'morality', 'morals', 'commercialism',
                   'conformation', 'conformity', 'context of use', 'dependableness', 'reliability', 'reliableness', 'equity', 'supervising', 'supervision', 'responsibleness',
                   'reliance', 'trustfulness', 'trustingness', 'confidence', 'trustiness', 'usefulness', 'attention', 'caution', 'precaution', 'concern', 'competency', 'availability',
                   'availableness', 'self respect', 'diverseness', 'variety', 'equivalence', 'fairness', 'humanness', 'justness', 'engagement', 'wellbeing', 'answerability',
                   'namelessness', 'understandability', 'consistence', 'reliableness', 'safe', 'protection', 'transparence', 'transparentness', 'usableness', 'useableness']
OtavioThesaurus = toLower(OtavioThesaurus)

TaxonomiaArtigo = ['Fairness', 'Bias', 'Safety', 'Harmful content', 'Toxicity', 'Hallucinations', 'Privacy', 'Data protection', 'Interaction risks', 'Security', 'Robustness',
                   'Cybercrime', 'Governance', 'Regulation', 'Labor displacement', 'Economic impact', 'Transparency', 'Explainability', 'Evaluation', 'Auditing', 'Sustainability', 
                   'Copyright', 'Authorship', 'Miscellaneous']
TaxonomiaArtigo = toLower(TaxonomiaArtigo)

ArtigoThesaurus = ['fairly', 'equity', 'preconception', 'safe', 'toxic', 'harm', 'risk', 'authoritativeness', 'sensitivity', 'misinformation', 'inaccuracies', 'privateness', 'secrecy',
                   'overreliance', 'manipulation', 'unethical actions', 'dependency', 'lack of empaty', 'protection', 'validity', 'hardiness', 'attack', 'fraud', 'administration',
                   'regularisation', 'regulating', 'effect', 'economical', 'transparence', 'transparentness', 'rating', 'valuation', 'audit', 'inspect', 'renewable energy',
                   'plagiarism', 'Accountability', 'responsibly', 'trustworthiness', 'moral views']
ArtigoThesaurus = toLower(ArtigoThesaurus)


#EthicalIssues = TaxonomiaArtigo

EthicalIssues = list(set(TaxonomiaArtigo + ArtigoThesaurus))

# Criando o DataFrame que guardará as ocorrências dos comentários
ocorrenciasComents = pd.DataFrame()
ocorrenciasComents['EthicalIssues'] = EthicalIssues
ocorrenciasComents['Ocorrencias'] = 0
ocorrenciasComents['PalavrasAntes'] = ""
ocorrenciasComents['PalavrasDepois'] = ""
ocorrenciasComents = ocorrenciasComents.set_index('EthicalIssues')

# Criando o DataFrame que guardará as ocorrências do título e descrição
ocorrenciasTituloDescricao = pd.DataFrame()
ocorrenciasTituloDescricao['EthicalIssues'] = EthicalIssues
ocorrenciasTituloDescricao['Ocorrencias'] = 0
ocorrenciasTituloDescricao['PalavrasAntes'] = ""
ocorrenciasTituloDescricao['PalavrasDepois'] = ""
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.set_index('EthicalIssues')

## **Contando a ocorrência dos termos:**

In [ ]:
# Função que faz a contagem das ocorrências dos termos e conta as palavras antes e depois
def contaOcorrencias(tokens, dfOcorrencias):
    for listaTokens in tokens:
        for index, token in enumerate(listaTokens):
            if token in dfOcorrencias.index:
                
                # Adicionando uma ocorrência do token
                dfOcorrencias.loc[token, 'Ocorrencias'] += 1
                
                if (index > 0):
                    # Adicionando a palavra anterior se existir
                    if (dfOcorrencias.loc[token, 'PalavrasAntes'] == ""):
                        dfOcorrencias.loc[token, 'PalavrasAntes'] = listaTokens[index - 1]
                    else:
                       dfOcorrencias.loc[token, 'PalavrasAntes']  += ("\n" + listaTokens[index - 1])
                    
                if (index < len(listaTokens) - 1):
                    # Adicionando a palavra depois se existir
                    if (dfOcorrencias.loc[token, 'PalavrasDepois'] == ""):
                        dfOcorrencias.loc[token, 'PalavrasDepois'] = listaTokens[index + 1]
                    else:
                       dfOcorrencias.loc[token, 'PalavrasDepois']  += ("\n" + listaTokens[index + 1])

contaOcorrencias(comentariosToK, ocorrenciasComents)
contaOcorrencias(tituloDescricaoToK, ocorrenciasTituloDescricao)

## **Deixando o dataframe no formato correto:**

In [ ]:
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.reset_index()
ocorrenciasComents = ocorrenciasComents.reset_index()

ocorrenciasTituloDescricao['PalavrasAntes'] = ocorrenciasTituloDescricao['PalavrasAntes'].str.split('\n')
ocorrenciasTituloDescricao['PalavrasDepois'] = ocorrenciasTituloDescricao['PalavrasDepois'].str.split('\n')

ocorrenciasComents['PalavrasAntes'] = ocorrenciasComents['PalavrasAntes'].str.split('\n')
ocorrenciasComents['PalavrasDepois'] = ocorrenciasComents['PalavrasDepois'].str.split('\n')

ocorrenciasComents = ocorrenciasComents[ocorrenciasComents['Ocorrencias'] > 0]
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao[ocorrenciasTituloDescricao['Ocorrencias'] > 0]

ocorrenciasComents = ocorrenciasComents.sort_values(by='Ocorrencias', ascending=False)
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.sort_values(by='Ocorrencias', ascending=False)

## **Criando uma tabela expansivel**

In [ ]:
# Função para criar a tabela com linhas expansíveis e contagem de palavras
def criarTabelaExpansivel(df):
    displayWidgets = []
    
    for _, linha in df.iterrows():
        # Conta as palavras em 'palavrasAntes' e 'palavrasDepois'
        antes_counts = Counter(linha['PalavrasAntes']).most_common()  # Ordena por contagem (decrescente)
        depois_counts = Counter(linha['PalavrasDepois']).most_common()  # Ordena por contagem (decrescente)
        
        # Cria uma linha com os dados principais
        linha_widget = widgets.HBox([
            widgets.Label(value=str(linha["EthicalIssues"]), layout=widgets.Layout(width="120px")),
            widgets.Label(value=str(linha["Ocorrencias"]), layout=widgets.Layout(width="60px")),
            widgets.Label(value="Clique para expandir", layout=widgets.Layout(width="200px"))
        ])
        
        # Cria o conteúdo expansível com `palavrasAntes` e `palavrasDepois` e suas contagens
        widgetDetalhes = widgets.VBox([
            widgets.Label(value=f"Palavras Antes: {', '.join([f'{word} ({count})' for word, count in antes_counts])}"),
            widgets.Label(value=f"Palavras Depois: {', '.join([f'{word} ({count})' for word, count in depois_counts])}")
        ])
        
        # Torna o conteúdo expansível
        expansao = widgets.Accordion(children=[widgetDetalhes])
        expansao.set_title(0, f"Detalhes de '{linha['EthicalIssues']}'")
        
        # Agrupa a linha principal e o conteúdo expansível
        linha_box = widgets.VBox([linha_widget, expansao])
        displayWidgets.append(linha_box)
    
    # Exibe a tabela com todas as linhas e detalhes
    display(widgets.VBox(displayWidgets))

## **Exibindo os resultados:**

In [ ]:
# Exibe a tabela expansível
criarTabelaExpansivel(ocorrenciasComents)


In [ ]:
# Exibe a tabela expansível
criarTabelaExpansivel(ocorrenciasTituloDescricao)

# **Exportando as Tabelas**

In [ ]:
# Exportando a tabela expansível como um arquivo html

def exportar_para_html(df, filepath):
    html = """<html>
    <head>
        <meta charset="UTF-8">
        <style>
            table {width: 100%%; border-collapse: collapse;}
            th, td {border: 1px solid black; padding: 8px; text-align: left;}
            th {background-color: #f2f2f2;}
            .detalhes {display: none;}
        </style>
        <script>
            function toggleDetalhes(id) {
                var elemento = document.getElementById(id);
                elemento.style.display = (elemento.style.display === "none") ? "block" : "none";
            }
        </script>
    </head>
    <body>
        <h2>Tabela Expansível</h2>
        <table>
            <tr>
                <th>Ethical Issues</th>
                <th>Ocorrências</th>
                <th>Detalhes</th>
            </tr>
    """

    for i, (_, linha) in enumerate(df.iterrows()):
        antes_counts = Counter(linha["PalavrasAntes"]).most_common()
        depois_counts = Counter(linha["PalavrasDepois"]).most_common()

        detalhes = f"""<p><strong>Palavras Antes:</strong> {', '.join([f'{word} ({count})' for word, count in antes_counts])}</p>
                       <p><strong>Palavras Depois:</strong> {', '.join([f'{word} ({count})' for word, count in depois_counts])}</p>"""

        html += f"""
            <tr>
                <td>{linha["EthicalIssues"]}</td>
                <td>{linha["Ocorrencias"]}</td>
                <td><button onclick="toggleDetalhes('detalhes{i}')">Expandir</button></td>
            </tr>
            <tr id="detalhes{i}" class="detalhes">
                <td colspan="3">{detalhes}</td>
            </tr>
        """

    html += "</table></body></html>"

    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)


In [ ]:
#exportar_para_html(ocorrenciasComents, "path_to_html")
#exportar_para_html(ocorrenciasTituloDescricao, "path_to_html")

# **Visualização dos Dados:**

## **Dataframes finais:**

In [ ]:
ocorrenciasComents

In [ ]:
ocorrenciasTituloDescricao

## **Visualizações:**

### **Gráfico de Barras Vertical (Comentários)**

In [ ]:
plt.figure(figsize=(8, 10))
sns.barplot(x="Ocorrencias", y="EthicalIssues", data=ocorrenciasComents.sort_values("Ocorrencias", ascending=False), palette="viridis")
plt.xlabel("Ocorrências")
plt.ylabel("Temas Éticos")
plt.title("Frequência de Ocorrências de Temas Éticos")
plt.tight_layout()
plt.show()

### **Gráfico de Barras Vertical (Infos)**

In [ ]:
plt.figure(figsize=(8, 10))
sns.barplot(x="Ocorrencias", y="EthicalIssues", data=ocorrenciasTituloDescricao.sort_values("Ocorrencias", ascending=False), palette="viridis")
plt.xlabel("Ocorrências")
plt.ylabel("Temas Éticos")
plt.title("Frequência de Ocorrências de Temas Éticos")
plt.tight_layout()
plt.show()

### **Palavras que Mais Ocorreram Antes e Depois (Comentários):**

In [ ]:

# Função corrigida para contar palavras
def getTopPalavra(lista_palavras, n=3):
    # Achatamos TODOS os níveis de listas aninhadas
    palavras = []
    for sublist in lista_palavras:
        if isinstance(sublist, list):
            palavras.extend(sublist)
        else:
            palavras.append(sublist)
    return Counter(palavras).most_common(n)


ocorrenciasComents['Antes'] = ocorrenciasComents['PalavrasAntes'].apply(lambda x: getTopPalavra(x, 1))
ocorrenciasComents['Depois'] = ocorrenciasComents['PalavrasDepois'].apply(lambda x: getTopPalavra(x, 1))

# DataFrame de resultados
result_df_coment = pd.DataFrame({
    'Problema Ético': ocorrenciasComents['EthicalIssues'],
    'Palavra Antes': ocorrenciasComents['Antes'].apply(lambda x: f"{x[0][0]} ({x[0][1]})" if x else '-'),
    'Palavra Depois': ocorrenciasComents['Depois'].apply(lambda x: f"{x[0][0]} ({x[0][1]})" if x else '-'),
    'Freq Antes': ocorrenciasComents['Antes'].apply(lambda x: f"{x[0][1]}" if x else '-'),
    'Freq Depois': ocorrenciasComents['Depois'].apply(lambda x: f"{x[0][1]}" if x else '-')
})

result_df_coment["Freq Antes"] = pd.to_numeric(result_df_coment["Freq Antes"], errors="coerce")
result_df_coment["Freq Depois"] = pd.to_numeric(result_df_coment["Freq Depois"], errors="coerce")


# Estilo da tabela
def style_table(styler):
    styler.set_properties(**{
        'text-align': 'left',
        'padding': '6px',
        'font-family': 'Arial',
        'border': '1px solid #ddd'
    })
    styler.set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#f8f9fa'), 
                 ('font-weight', 'bold'),
                 ('color', '#333')]
    }])
    return styler

# Exibição
display(HTML(
    result_df_coment[["Problema Ético", "Palavra Antes", "Palavra Depois"]].style
    .pipe(style_table)
    .set_caption("<b>Palavras mais frequentes associadas a cada problema ético</b>")
    .to_html()
))

### **Palavras que Mais Ocorreram Antes e Depois (Infos):**

In [ ]:
# Função corrigida para contar palavras
def getTopPalavra(lista_palavras, n=3):
    # Achatamos TODOS os níveis de listas aninhadas
    palavras = []
    for sublist in lista_palavras:
        if isinstance(sublist, list):
            palavras.extend(sublist)
        else:
            palavras.append(sublist)
    return Counter(palavras).most_common(n)


ocorrenciasTituloDescricao['Antes'] = ocorrenciasTituloDescricao['PalavrasAntes'].apply(lambda x: getTopPalavra(x, 1))
ocorrenciasTituloDescricao['Depois'] = ocorrenciasTituloDescricao['PalavrasDepois'].apply(lambda x: getTopPalavra(x, 1))

# DataFrame de resultados
result_df_info = pd.DataFrame({
    'Problema Ético': ocorrenciasTituloDescricao['EthicalIssues'],
    'Palavra Antes': ocorrenciasTituloDescricao['Antes'].apply(lambda x: f"{x[0][0]} ({x[0][1]})" if x else '-'),
    'Palavra Depois': ocorrenciasTituloDescricao['Depois'].apply(lambda x: f"{x[0][0]} ({x[0][1]})" if x else '-'),
    'Freq Antes': ocorrenciasTituloDescricao['Antes'].apply(lambda x: f"{x[0][1]}" if x else '-'),
    'Freq Depois': ocorrenciasTituloDescricao['Depois'].apply(lambda x: f"{x[0][1]}" if x else '-')
})

result_df_info["Freq Antes"] = pd.to_numeric(result_df_info["Freq Antes"], errors="coerce")
result_df_info["Freq Depois"] = pd.to_numeric(result_df_info["Freq Depois"], errors="coerce")

# Estilo da tabela
def style_table(styler):
    styler.set_properties(**{
        'text-align': 'left',
        'padding': '6px',
        'font-family': 'Arial',
        'border': '1px solid #ddd'
    })
    styler.set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#f8f9fa'), 
                 ('font-weight', 'bold'),
                 ('color', '#333')]
    }])
    return styler

# Exibição
display(HTML(
    result_df_info[["Problema Ético", "Palavra Antes", "Palavra Depois"]].style
    .pipe(style_table)
    .set_caption("<b>Palavras mais frequentes associadas a cada problema ético</b>")
    .to_html()
))

In [ ]:
# Tabela com cores Info
styled_df = result_df_info.style.set_properties(subset=["Palavra Antes"], **{'color': 'blue', 'font-weight': 'bold'}) \
                    .set_properties(subset=["Palavra Depois"], **{'color': 'red', 'font-weight': 'bold'})
styled_df

In [ ]:
# Tabela com cores Coments
styled_df = result_df_coment.style.set_properties(subset=["Palavra Antes"], **{'color': 'blue', 'font-weight': 'bold'}) \
                    .set_properties(subset=["Palavra Depois"], **{'color': 'red', 'font-weight': 'bold'})
styled_df

In [ ]:
# Gráfico de barras horizontais
fig, ax = plt.subplots(figsize=(8, 6))
df = result_df_coment.head(5)
y = np.arange(len(df))
bar_width = 0.2

# Barras
ax.barh(y - bar_width/2, df["Freq Antes"], height=bar_width, color='skyblue', label='Antes')
ax.barh(y + bar_width/2, df["Freq Depois"], height=bar_width, color='salmon', label='Depois')

# Rótulos com palavras
for i in range(len(df)):
    ax.text(df["Freq Antes"].iloc[i] + 1, y[i] - bar_width/2, df["Palavra Antes"].iloc[i], va='center', color='blue')
    ax.text(df["Freq Depois"].iloc[i] + 1, y[i] + bar_width/2, df["Palavra Depois"].iloc[i], va='center', color='red')

# Estilo
ax.set_yticks(y)
ax.set_yticklabels(df["Problema Ético"])
ax.invert_yaxis()
ax.set_xlabel("Frequência")
ax.set_title("Palavras Antes e Depois dos Problemas Éticos")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de barras horizontais
fig, ax = plt.subplots(figsize=(8, 10))
df = result_df_info.head(10)
y = np.arange(len(df))
bar_width = 0.2

# Barras
ax.barh(y - bar_width/2, df["Freq Antes"], height=bar_width, color='skyblue', label='Antes')
ax.barh(y + bar_width/2, df["Freq Depois"], height=bar_width, color='salmon', label='Depois')

# Rótulos com palavras
for i in range(len(df)):
    ax.text(df["Freq Antes"].iloc[i] + 1, y[i] - bar_width/2, df["Palavra Antes"].iloc[i], va='center', color='blue')
    ax.text(df["Freq Depois"].iloc[i] + 1, y[i] + bar_width/2, df["Palavra Depois"].iloc[i], va='center', color='red')

# Estilo
ax.set_yticks(y)
ax.set_yticklabels(df["Problema Ético"])
ax.invert_yaxis()
ax.set_xlabel("Frequência")
ax.set_title("Palavras Antes e Depois dos Problemas Éticos")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
from wordcloud import WordCloud

# Dicionários com as palavras e frequências
freq_antes = dict(zip(result_df_coment["Palavra Antes"], result_df_coment["Freq Antes"]))
freq_depois = dict(zip(result_df_coment["Palavra Depois"], result_df_coment["Freq Depois"]))

# Gerar nuvens
wordcloud_antes = WordCloud(width=400, height=300, background_color='white', colormap='Blues').generate_from_frequencies(freq_antes)
wordcloud_depois = WordCloud(width=400, height=300, background_color='white', colormap='Reds').generate_from_frequencies(freq_depois)

# Mostrar lado a lado
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
axs[0].imshow(wordcloud_antes, interpolation='bilinear')
axs[0].set_title("Palavras Antes")
axs[0].axis('off')

axs[1].imshow(wordcloud_depois, interpolation='bilinear')
axs[1].set_title("Palavras Depois")
axs[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from wordcloud import WordCloud

# Dicionários com as palavras e frequências
freq_antes = dict(zip(result_df_info["Palavra Antes"], result_df_info["Freq Antes"]))
freq_depois = dict(zip(result_df_info["Palavra Depois"], result_df_info["Freq Depois"]))

# Gerar nuvens
wordcloud_antes = WordCloud(width=400, height=300, background_color='white', colormap='Blues').generate_from_frequencies(freq_antes)
wordcloud_depois = WordCloud(width=400, height=300, background_color='white', colormap='Reds').generate_from_frequencies(freq_depois)

# Mostrar lado a lado
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
axs[0].imshow(wordcloud_antes, interpolation='bilinear')
axs[0].set_title("Palavras Antes")
axs[0].axis('off')

axs[1].imshow(wordcloud_depois, interpolation='bilinear')
axs[1].set_title("Palavras Depois")
axs[1].axis('off')

plt.tight_layout()
plt.show()